# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**20/04/2020 - C8 S4**

## Manejo de Datos 2 y Visualización

Es Habitual que la información se encuentre dispersa en distintas fuentes y que por tanto, se requiera unificar el contenido. Si bien esto representa una componente importante del análisis de datos, el proceso no termina ahí, pues es necesario agregar, categorizar, agrupar y aplicar funciones a las agrupaciones logradas. Tales funciones se traducen en calculo de estadísticas, uso de tablas pivote, generación de visualizaciones entre otras tareas. 

En está cátedra, nos centraremos en las herramientas que facilitan los procesos de unificación, agrupación y visualización presentes en el flujo de trabajo de ciencia de datos. Tales herramientas se cimientan en `pandas` y `matplotlib`. 

### Unificación de datasets

La información contenida en los objetos `Series` y `DataFrame` puede ser combinada de distintas maneras. Una de ellas es por medio de *merging* o *fusión*, este tipo de operación permiten combinar datasets al enlazar filas según uno o más campos 'llave'. 

La función `merge` de pandas permite implementar estos procedimientos.

**Ejemplo**

Se generan 2 objetos `DataFrame` 

In [ ]:
import pandas as pd


df1 = pd.DataFrame({'llaves': list('abdabhakl'), 'data': range(9)})

# Se etiquetan los indices (ind) y columnas (columns)
df1.index.name = 'ind' 
df1.columns.name ='columns'

df1.sort_values(by = 'llaves', inplace=True)
df1

Se obtiene un nuevo dataframe por reindex del primero

In [ ]:
df2 = df1.reindex(index = [1,3,5])

# Se alteran cierto valores
df2.data = [10, 30, 50]

df2

En este caso se estudia la relación *muchos a uno* (*many to one*). Acá `df1` posee 3 instancias de la llave 'a' y 2 de 'b', por su parte `df2` posee solo una copia de cada 'llave'. Al utilizar `merge` sobre las llaves se obtiene:

In [ ]:
pd.merge(df1,df2, on = 'llaves')

En este caso tanto `df1` como `df2` comparten la columna `llaves` y merge procese a unificar ambos datasets utilizando la columna llave como punto de union. Observe que se preservan los valores de `df1` en `data_x` y de `df2` en `data_y`. 

Si los dataframes a fusionar no poseen columnas en común, es posible entregar los argumentos `left_on` para la llave a utilizar en el dataframe izquierdo (como argumento de `merge`), de manera análoga se debe introducir `rigth_on`. 

Se crean dos nuevos dataframes sin columnas en común

In [ ]:
df3 = df1.copy()

# Se generan las nuevas llaves
df3['l_llave'] = df3.llaves
df3['data_1'] = df3.data

# Se eliminan las columnas antiguas
df3.drop(['data','llaves'], axis = 1, inplace = True)


# Se repite el proeso para df4

df4 = df2.copy()
df4['r_llave'] = df2.llaves
df4['data_2'] = df2.data
df4.drop(['data','llaves'], axis = 1, inplace = True)

Se procede a fusionar `df3` y `df4`

In [ ]:
pd.merge(df3,df4, left_on = 'l_llave', right_on= 'r_llave')

Recuperando (salvo suplcación de llave) el dataframe anterior. `merge` opera por defecto haciendo uniones *internas* (*inner join*), esto significa, hay llaves en alguna dataset sin valor asociado en el otro, se omiten.

**Ejemplo**

Se comprueba lo anterior, para ello se agregan dos nuevas filas a `df3`. Observe que la opción `ignore_index` permite agregar diccionarios al final de `df3`

In [ ]:
df3 = df3.append({'data_1':'y', 'l_llave':100}, ignore_index=True)
df3 = df3.append({'data_1':'z', 'l_llave':200}, ignore_index=True)
df3.tail(2)

Se repite el proceso de unión

In [ ]:
pd.merge(df3,df4, left_on = 'l_llave', right_on= 'r_llave')

Lo que comprueba el resultado. 

**Ejercicio**

1. Utilice los objetos `DataFrame` `df3` y `df4` para explorar las opciones de `pd.merge(how = opt)` donde `opt` pertenece a `{'left', 'right', 'outer', 'inner'}`. 


El tipo de unón *muchos a muchos* (*many to many*).Se comporta de manera distinta. En este caso, se genera un producto cartesiano de las filas. 

**Ejemplo** 

Se agregan valores a `df4`, para ello se crea un daaframe con los valores a agregar, se declaran sus columnas y se agrega al final de `df4`. Observe la opción `ignore_index = True` esta permite omitir los índices de `df_pivot` y continuar con los de `df4`.

In [ ]:
df_pivot = pd.DataFrame(
    [['a', 300], ['a', 301], ['h', 500], ['h', 501], ['b', 100]],
    columns=df4.columns)

# Observe que append en dataframes no opera como en listas (no es inplace)
df4 = df4.append(df_pivot, ignore_index=True)
df4.sort_values(by = 'r_llave', inplace=True)
df4

Se procede a unir `df3` con `df4`

In [ ]:
pd.merge(df3,df4, left_on='l_llave', right_on='r_llave', how= 'inner')

Como hay 3 instancias de 'a' en `df3` y 3 instancias de 'a' en `df4`, se tienen un total de 9 entradas relacioandas a 'a' en la unión (desde el índice 0 al 8). Esto se puede intepretar de la siguiente manera: 

Cuando hay **una** columna de llaves, es decir, independiente del nombre de las columnas `left_on` y `right_on`, se escoge una por dataset. Se tendrá en este caso, que la  unión **muchos a muchos** repetirá los identificadores de la columna de llaves, tantas veces como lo requiera el producto cartesiano de los valores asociados a tales llaves. Por tal motivo, en el ejemplo anterior, aparecen 9 instancias de 'a'.

Por otra parte, `pd.merge()` permite la unión de datasets proporcionando **más de una** columna llave por dataset. En este caso, los identificadores a utilizar, pasan a ser elementos del producto cartesiano entre los identificadores de cada columna de llaves proporcionada, los cuales (identificadores), serán repetidos, tantas veces como el producto cartesiano de sus valoes asociados lo requiera. 


**Ejemplo**

Se estudia la unión de datasets *muchos a muchos* proporcionando múltiples  columnas de llaves. Para ello se definen los siguientes dataframes

In [ ]:
df_m = pd.DataFrame({
    'llave_1': ['a', 'a', 'b'],
    'llave_2': [5, 7, 5],
    'data_m': [10, 20, 30]
})
df_m

In [ ]:
df_n = pd.DataFrame({
    'llave_1': ['a', 'a', 'b', 'b'],
    'llave_2': [5, 5, 5, 7],
    'data_n': [100, 200, 300, 400]
})
df_n

Se procede a unir ambos datasets

In [ ]:
pd.merge(df_m,df_n, on = ['llave_1','llave_2'], how = 'outer')

Al elegir `how = 'outer'` se realiza una *unión externa*, a diferencia de la unión interna, acá se agregan valores nulos cuando no se tiene la información (en vez de omitr la fila).


En este ejemplo se aprecia que el producto cartesiano entre `llave_1` y `llave_2` corresponde a: 

 * 'a',5
 * 'a',7
 * 'b',5
 * 'b',7
 
Se observa que para el par 'a',5  el producto cartesiano de sus valores asociados es:

* 10, 100
* 10, 200

mientras que para 'a',7 solo existe un valor en `df_m` y es nulo en `df_n`. Se continua con dicha lógica hasta completar la tabla.

**Ejercicios**

1. Cargue los datos `'data/timesData.csv'` en la variable `df` indexe por año y ranking,

2. Genere una función `data_process(df, year,n)` que procese al dataset `df` de manera tal, que retorne un sub-dataset con las primeras `n` universidades (según ranking), con las columnas 'university_name', 'teaching' y 'research'. Además debe tener un indice múlti - nivel, donde 'country' sea el nivel más externo y 'wolrd_rank' se el nivel interno. *Hint*: Investigue el método `.set_index()`.

3. Utilice `data_process()` para generar datasets de los años 2011 y 2016 para `n=20` cada uno.

4. Haga una unión interna entre ambos datasets según los multi - indices, el resultado de esto le permitirá saber que universidades han mantenido su puesto, por país, al tiempo que puede verificar modificaciones en los puntajes de enseñanza e investigación. Observe que para esto deberá utilizar los argumentos `left_index` y `right_index`. Finalmente deduzca que la unión de datasets por indexación múltiple, obedece e mismo principio que la unión por múltiples columnas.

Otro tipo de unificación de datos es la concatenación, gracias a la indexación de pandas, es posible concatenar objetos con distintas indexaciones por medio de uniones internas (solo mantener datos ejes en común), elegir ejes de concatenación, concatenar *slices* de dataframes para componer otro, etc.

La función `pd.concat()` permite todo lo anterior y su funcionamiento es bastante intuitivo.

**Ejemplo** 

Se estudia la concatenación en series, para ello se definen serie_1, serie_2 y serie_3.

In [ ]:
serie_1 = pd.Series([1,2,3], index= list('abc'))
serie_2 = pd.Series([4,5], index= list('cd'))
serie_3 = pd.Series([6,7,8,9], index= list('efgh'))

`pd.concat()` actua de la manera esperada en este caso

In [ ]:
pd.concat([serie_1,serie_2,serie_3])

Observe que se entrega una lista con las series como argumento. `pd.concat()` permite seleccionar eje de concatenacion. En este caso, el valor por defecto es `join = 'outer`

In [ ]:
# Se entrega el argumento sort=False para evitar un warning inocuo
pd.concat([serie_1,serie_2,serie_3], axis=1, sort=False)

Si por otra parte se entrega el parámetro `join ='inner'`, se tendrá la intersección de las 3 series (que es nula).

In [ ]:
pd.concat([serie_1,serie_2,serie_3], axis=1, sort=False, join = 'inner')

Si por otra parte, solo se concatenan la series 1 y 2

In [ ]:
pd.concat([serie_1,serie_2], axis=1, sort=False, join = 'inner')

Se obtiene lo esperado. 

**Ejercicios**

1. El argumento `keys` de la función `pd.concat()` permite crear un dataframe con multi - indices. Utilice las series `serie_1`, `serie_2` y `serie_3` y `keys` con el argumento `['s1', 's2, 's3']`. Observe que al concatenar por columnas, las llaves anteriores pasan a ser etiquetas de columna.

2. Utilice los dataframes `df_m` y `df_n` de la sección pasada. Utilice el parámetro `keys = [df_m, df_n]`, genere un nuevo dataframe:
    1. Concatenando por fila para unión interna y externa. 
    2. Concatenado por columna para unión interna y externa
¿Como cambia el papel del parámetro `keys`?

### Elementos de Visualización en Python

Lograr visualizaciones (plots) informativas es de gran importancia en análisis de datos. Su aplicación en este campo es transversal, abarcando desde la parte exploratoria, para por ejemplo, identificación de outliers o búsqueda de transformaciones en las variables. Por otra parte, las visualizaciones son una manera natural de mostrar los resultados obtenidos y facilitar la interpretación de los fenómenos estudiados. En Python existen librerías capaces de producir visualizaciones de manera intuitiva tanto estáticas como dinámicas, en este ámbito, la librería raíz es `matplotlib`. Tal librería está diseñada para la creación de visualizaciones de alta calidad para publicaciones. 


`Matplotlib` puede exportar visualizaciones en formatos vectorizados y comprimidos como son PDF, SVG, JPG, PNG, GIF , entre otros. Existe una extensa variedad de librerías gráficas basadas en `matplotlib`, una de ellas es `seaborn`, la cual también se explora en este capítulo.

# Importación y primeros pasos<a id="pp"></a> 

Como con NumPy y Pandas, existe una convención a la hora de importar esta librería:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
data = np.arange(10)
plt.plot(data)

Para trabajar de manera interactiva, la linea mágica:
```
%matplotlib notebook

```
Toma provecho de las capacidades del proyecto Jupyter.

In [ ]:
%matplotlib notebook
plt.plot(data)

## Figuras y subplots

Las visualizaciones `plots` en matplotlib actúan dentro de un objeto más general denominado figura o `figure`. Es posible crear un objeto `figure` vacío para luego agregar las visualizaciones deseadas a través de:

```python
fig = plt.figure()

```
Esto genera una ventana vacía, no obstante, es posible modificar opciones del objeto `fig` declarado anteriormente. Dentro de estas opciones se encuentra `figsize` la cual permite indicar el tamaño y relación de aspecto al momento de almacenar la visualización en el disco.

Para insertar visualizaciones dentro de la figura recién creada, es necesario añadir un `subplot`, el método `add_subplot` permite agregar múltiples visualizaciones a la figura en la cual se invoca.

In [ ]:
fig = plt.figure()

In [ ]:
ax1 = fig.add_subplot(2,2,1)

El comando anterior indica que la figura tiene una estructura que comprende 2 visualizaciones por fila y 2 por columna, es decir la figura comprenderá 4 subfiguras de las cuales se selecciona la primera y se almacena como `ax1`.

**Ejercicio**

* Agregue 2 subfiguras a la figura `fig`. estas deben estar en la esquina superior derecha e inferior izquierda. Denote estas subfiguras como `ax2` y `ax3`.

Observación: En Jupyter, los plots se reinician luego de cada evaluación de celda, por lo tanto para obtener el esquema de subfiguras solicitado, debe declarar la figura y todos los subplots en la misma celda.

Al agregar un comando de visualización como por ejemplo:
```pyhton
plt.plot([-1,1,-1,1])
```
matplotlib modificará la última figura creada en su último subplot, en caso de ser una figura vacía, se creará un nuevo subplot en ella. Por lo tanto, al evaluar el código anterior, se deberá agregar una figura con forma de "N" en el último subplot del ejercicio anterior.

In [ ]:
plt.plot([-1,1,-1,1])

Por otra parte, es posible añadir visualizaciones en los demás subplots usando las variables antes almacenadas:

In [ ]:
ax2.plot(np.random.randn(10), 'r--')

En tal caso, se indicó generar el gráfico de una variable aleatoria normal estándar, de la cual se extraen 10 muestras. El comando `r--` indica que el gráfico debe ser rojo, con líneas "cortadas" y posicionado en el subplot `ax2`.

**Ejercicio**

* Genere un arreglo aleatorio de dimensión 100, explore con tal arreglo  el método `hist`. ¿Qué función cumple el parámetro `alpha`?

* Estudie el método `scatter`. ¿En qué se diferencia al método `plot`?


Otra manera de obtener figuras compuestas de subplots es a través del método `plt.subplots`, este entrega como `return` una figura y un arreglo NumPy con los objetos `subplot` en su interior:
```python
fig, axes = plt.subplots(2,3)
```

In [ ]:
fig, axes = plt.subplots(2,3)

In [ ]:
axes

En cuanto a la variable `axes` esta presenta la facilidad de ser accedida como un arreglo de dos dimensiones, por ejemplo:

In [ ]:
axes[0,1]

permite acceder al primer subplot de la segunda columna de la figura `fig`.

**Ejercicio**

* Utilice la notación anterior para graficar solo en los subplots correspondientes a la esquina inferior izquierda y esquina superior derecha.

* Una opción bastante útil es la de mantener la misma escala en cada subplot de la figura creada. Explore las opciones `sharex` y `sharey`. ¿Cómo se comporta matplotlib al no indicar estas opciones ?

## Ajustes de espaciado en subplots

Por defecto, matplotlib maneja de manera automática el espacio alrededor de cada subplot. Esto se hace de manera relativa y en función a las dimensiones de la figura, sin embargo, es posible modificar esta opción utilizando `subplots_adjust` en objetos del tipo `Figure`:

```python
subplots_adjust(left=None, bottom=None, right=None, top=None,wspace=None, hspace=None)
``` 

En este contexto, las variables `wspace` y `hspace` controlan el porcentaje de ancho (width) y alto (height) que se debe utilizar entre subplots. 

**Ejercicio**

* Genere una figura con 4 histogramas, todas las subfiguras deben estar adyacentes unas con otras (no debe haber espacio entre ellas). (Hint: genere los histogramas usando 2 ciclos `for` anidados)

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True)
for i in range(2):
    for j in range(2):
        axes[i, j].hist(np.random.randn(500), color='k', alpha=0.5)
plt.subplots_adjust(wspace=0, hspace=0)

## Colores, marcadores y estilos de línea

La función de visualización principal de matplotlib `plot` permite ingresar arreglos de coordenadas `x` e `y`, en este caso, para producir un gráfico color verde con líneas o "dashes" se puede ingresar un arreglo `X` y un arreglo `Y` ambos de la misma dimensión y ejecutar:

```python
ax.plot(X,Y,'g--')
```

Donde `ax` es un subplot antes declarado. La notación anterior es una versión comprimida de las opciones `linesyle='--'` y `color='g'` de tal manera que el comando anterior equivale a ingresar:

```python
ax.plot(X,Y,linestyle = '--', color = 'g')
```

**Ejericio** 

* Compruebe lo antes descrito.

In [ ]:
X = np.arange(10)
Y = X + np.random.randn(10)

fig = plt.plot(X,Y,linestyle = '--', color = 'g')

En matplotlib es posible personalizar los colores que se desea obtener en un gráfico, los colores más usados poseen su propia abreviación por ejemplo `'g'` se refiere a verde, `'k'` a negro, `'r'` a rojo, etc ...

Si se desea ser aún más especifico, es posible ingresar un código hex para el color que se desea visualizar:

**Ejercicio**

* Cambie el color del gráfico anterior por uno en código hex de su elección.

Adicionalmente los plots pueden tener *marcadores* para resaltar las coordenadas correspondientes a los datos. Esto, debido a que la función `plot` genera una línea continua entre los puntos que se ingresan, por tanto, puede aparecer la necesidad de especificar las ubicaciones de los datos reales proporcionados. Los marcadores se acceden a través de la opción `marker` de la función `plot`, la siguiente [referencia](https://matplotlib.org/api/markers_api.htmlreferencia) proporciona los marcadores disponibles. Se puede acceder también a la notación simplificada utilizando por ejemplo:

```python
plt.plot(np.random.randn(30).cumsum(), 'ko-')
```
Donde el marcador es 'o' el color es negro y el tipo de línea es continuo.

In [ ]:
plt.plot(np.random.randn(30).cumsum(), 'ko-')

Como se puede observar, la interpretación entre los puntos del gráfico es hecha de manera lineal, esto se puede personalizar con la opción `drawstyle`, la siguiente [referencia](https://matplotlib.org/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D.set_drawstyle) muestra los tipos de interpolación disponibles. 

En el siguiente ejemplo se especifica además la etiqueta o `label` de cada "línea" graficada:

In [ ]:
data = np.random.randn(30).cumsum()

plt.plot(data, 'ro--', label='Default')
plt.plot(data, 'k-' , drawstyle='steps-post', label='steps-post')
plt.legend()

Al final del ejemplo se ejecuta el comando `legend`, el cual indica que se de deben mostrar los objetos del tipo `label` declarados con la opción `label` en la función `plot`. En el caso de graficar sobre un subplot `ax` se puede llamar el método `ax.legend`.

## Ejes, etiquetas y leyendas


Para personalizar los valores mostrados en el eje `x`, es posible usar las opciones `set_xticks` y `set_xticklabels`:

In [ ]:
# Genera una serie aleatoria

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(np.random.randn(500).cumsum())

# Selecciona los puntos de interes en el eje x
ticks = ax.set_xticks([0, 36,150, 225, 453])


# Cambia los nombres de los valores en el eje

labels = ax.set_xticklabels(['cero','A','B','X','W'], rotation=30, fontsize='large')

Es también posible agregar un titulo a la visualización, para ello se llama la función`set_title`:

In [ ]:
ax.set_title('Titulo de prueba')

**Ejercicio**

* Utilice el método `set_xlabel` para dar un nombre al eje `x` del plot anterior.


Modificar el eje `y` consiste en el mismo procedimiento antes descrito.

 Los objetos definidos por el método `add_subplots` se denominan `axes`, estos, como clase, poseen el método `set` que permite ingresar sus propiedades a través de un diccionario. En el caso anterior es posible configurar el subplot `ax` por medio de:


In [ ]:
ops = {
    'title':'Titulo de prueba',
    'xlabel':'Ejercicio'
}

ax.set(**ops)

Se debe indicar el diccionario antes descrito utilizando como llaves los campos que se deseen ingresar, además se debe utilizar doble asterisco `**` al llamar el método `set`, esto pues la declaración de tal método recibe un diccionario con variables "identificadas por nombre" como entrada. En declaración de funciones esto se denota con el uso de variables `**kwargs`.

## Leyendas

Las leyendas corresponden al elemento principal de identificación de gráficos. Anteriormente se utilizó esta opción, en esta sección se estudia con más detalle.

La manera más sencilla de agregar una leyenda, cosiste en primero crear una etiqueta pasando la opción `label` al crear la visualización de interés para luego llamar el método `legend`.

In [ ]:
%matplotlib notebook 
from numpy.random import randn
fig = plt.figure(); ax = fig.add_subplot(1, 1, 1)

ax.plot(randn(100).cumsum(),'r'  , label='G_1')
ax.plot(randn(100).cumsum(),'k--', label='G_2')
ax.plot(randn(100).cumsum(),'g.-' , label='G_3')

ax.legend()

Como es de esperar, el método `legend` tiene opciones adicionales para, por ejemplo, controlar la posición de las legendas.

In [ ]:
ax.legend?

## Anotaciones y figuras sobre un subplot

Hasta ahora, las visualizaciones generadas corresponden a figuras más bien estándar. En caso de querer agregar anotaciones adicionales, consistentes de texto, flechas u otras formas, es posible usar los métodos `text`, `arrow` y `annotate`. 

`text` genera un texto en las coordenadas (x,y) a elección, este método también presenta opciones extras sobre el estilo:

```python
ax.text(10,5,'Anotación Cepal', family='monospace', fontsize='10')
```

In [ ]:
ax.text(70,5,'Anotación Cepal', family='monospace', fontsize='10')

Las anotaciones pueden también dibujar flechas con texto informativo. 

Además de añadir anotaciones, es posible agregar figuras, las figuras principales se denominan `patches`, alguno de estos son los rectángulos y círculos, en la documentación de `matplotlib.patches` se hace una recopilación de estos. A continuación se muestra un ejemplo:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
rect = plt.Rectangle((0.2, 0.75), 0.4, 0.15, color='k', alpha=0.3)
circ = plt.Circle((0.7, 0.2), 0.15, color='b', alpha=0.3)
pgon = plt.Polygon([[0.15, 0.15], [0.35, 0.4], [0.2, 0.6]],
color='g', alpha=0.5)
ax.add_patch(rect)
ax.add_patch(circ)
ax.add_patch(pgon)

## Guardado de figuras

Es posible almacenar en el disco las figuras producidas, para ello, el método `plt.savefig` es usado. Por ejemplo, para almacenar una figura en formato `SVG` con el nombre `figura_text` se puede hacer con comando:
```python
plt.savefig('.../ruta/figura_test.SVG')

```

El tipo de archivo se infiere automáticamente de la extensión usada, los tipos de archivos soportados incluyen `png` y `pdf` entre otros. Algunas opciones de bastante utilidad son `dpi` y `bbox_inches`, el siguiente comando permite guardar un figura en formato `png` con 400 dpi y con el mínimo espacio alrededor del gráfico:

```python
plt.savefig('figpath.png', dpi=400, bbox_inches='tight')
```

## Configuración de matplotlib

El paquete descrito hasta acá viene preconfigurado con esquemas de colores. Todos estos esquemas "por defecto" pueden ser personalizados extendiendo los parámetros globales de la librería. El método `rc`  permite hacer esto, por ejemplo, para cambiar el tamaño de las figura a 10x10 se puede ingresar:

```python
plt.rc('figure', figsize=(10,10))
```

El primer argumento de `rc` es el componente que se desea personalizar, en este caso es `figure`, se puede acceder también a `axes`, `xtick`, `ytick`, `grid`, `legend` entre otros. Luego se ingresan los parámetros que se deseen modificar, esto puede ser hecho a través de diccionarios:

```python
opt = {'family' : 'monospace',
       'weight' : 'bold',
       'size' : 'small'}
plt.rc('font', **opt)
```

# Visualizaciones con Pandas y seaborn<a id="pdSns"></a>

matplotlib se puede interpretar como una herramienta de "bajo nivel" para la producción de visualizaciones, es decir, esta se generan a partir de sus componentes básicos.

En Pandas por otra parte, los DataFrames pueden tener múltiples columnas además de etiquetas para las filas. Este paquete posee métodos de visualización incluidos, pensados para simplificar el proceso de producción a partir de Series y DataFrames. 

Otra librería basada en tal principio es `seaborn`, esta corresponde a una librería de gráficos estadísticos construida sobre matplotlib. A continuación se estudian las posibilidades de visualización tanto en pandas como en seaborn.

## Gráficos de línea

Las Series y Dataframes poseen atributo `plot`, este permite hacer gráficas simples, por defecto, este atributo genera gráficos de línea interpolados:

In [ ]:
fig = plt.figure()
s = female['PSU Average']
s.plot()

Los indices del objeto serie son interpretados como el eje `x`, esto se puede evitar con `use_index = False`. El los puntos de interés en `x` o "ticks" y sus limites se pueden ajustar por medio de `xticks` y `xlim`, de la misma manera se puede ajustar el eje `y`. Las opciones disponibles se pueden acceder en la documentación del `plot`.

Por lo general, lo métodos de visualización en pandas permiten un parámetro `ax` el cual puede ser un subplot de matplotlib. 

El método `plot` en DataFrames genera un plot distinto para cada visualización produciendo una "linea" distinta por columna además de generar una leyenda de manera automática.

In [ ]:
H = data.pivot_table(index=['Year', 'Sex']).unstack(1)
H.columns = ['female', 'male']

Data = H

In [ ]:
Data.plot()

Por su parte, el método `plot` de un DataFrame, posee opciones diferentes a las de una Serie, siendo `subplots` una de estas:

In [ ]:
Data.plot(subplots=True)

## Gráficos de barra

La función `plot.bar()` y `plot.barh()` generan gráficos de barra verticales y horizontales respectivamente. En esta caso, los índices ya sean de la Serie o DataFrame serán utilizados como el eje `x` (bar) o `y` (barh).

In [ ]:
fig, axes = plt.subplots(2, 1)
data = pd.Series(np.random.rand(16), index=list('abcdefghijklmnop'))

data.plot.bar(ax=axes[0], color='k', alpha=0.7)
data.plot.barh(ax=axes[1], color='k', alpha=0.7)

Las opciones `color = 'k'` y `alpha=0.7` proporcionan tanto el color como la transparencia de las barras.

In [ ]:
df = pd.DataFrame(np.random.rand(6, 4),

index=['one', 'two', 'three', 'four', 'five', 'six'],
columns=pd.Index(['A', 'B', 'C', 'D'], name='Titulo - personalizado'))

df.plot.bar()

Para crear gráficos de barra superpuestos a partir de un DataFrame la opción `stacked=True` lo permite de manera sencilla:

In [ ]:
df.plot.barh(stacked=True, alpha=0.9)

**Ejercicio** 

* Genere un gráfico de barras horizontal, donde se muestren las frecuencias para los puntajes promedio PSU obtenidos por las mujeres separados en 5 grupos (o bins). (Hint: `value_counts`)

In [ ]:
fig=plt.figure()
s_f = Data['female']
s_f.value_counts(bins=4).plot.barh()

Para hacer visualizaciones que requieran un agrupamiento previo exhaustivo de los datos, es posible usar `seaborn`:

In [ ]:
import seaborn as sns
fig=plt.figure()

sns.barplot(data=Data)

El argumento de barplot corresponde a un DataFrame u observaciones de un base de datos en arreglos `x` e `y`. En este caso, el gráfico corresponde a los valores promedio de los puntajes PSU tanto de hombres como de mujeres, además agrega un intervalo de confianza del 95% al valor (lineas negras).

Los barplots de seabron tiene además una opción denominada `hue` la cual permite separar a través de una nueva variable categórica. El siguiente ejemplo hace uso de esta opción utilizando los datos de efectividad en la educación básica obtenidos de la API DataChile.

In [ ]:
# https://es.datachile.io/geo/chile#education 
# Efectividad en educación:

import pandas as pd
df_efect = pd.read_csv('./datos/C3/efectividad.csv')

# Limpieza inicial

df_efect.replace('Educación especial', np.NaN)
df_efect.dropna(inplace=True)

In [ ]:
data_1 = pd.DataFrame(data=df_efect, columns=['Stage 1a','Avg efectiveness'])
data_1['bin'] = pd.cut(data_1['Avg efectiveness'], bins=5)

In [ ]:
fig = plt.figure()
sns.barplot(x=data_1.bin, y=data_1['Avg efectiveness'], hue=data_1['Stage 1a'])

Es posible cambiar el estilo de los gráficos usando el método `set` de seaborn:

In [ ]:
sns.set(style='whitegrid')

## Histogramas y gráficos de densidad

Un histograma es un tipo de gráfico de barras que proporciona un valor discretizado de las frecuencias. Para esto, se discretizan los valores presentes en la base de datos para luego ser separados en "bins", posteriormente se gráfica el número de veces que un bin es "poblado" por algún elemento de la base.

In [ ]:
data_1.plot.hist(bins=50)

Un tipo relacionado de datos son los gráficos de densidad, estos calculan una estimación de la distribución de probabilidad que pudo haber generado los datos. El procedimiento usual para esto, es aproximar tal distribución por medio de una mezcla de "kernels", es decir, distribuciones más simples. Por tanto, los gráficos de densidad son conocido con el nombre de estimadores de densidad por kernels o KDE por sus siglas en ingles. Usando la función `plot.kde`  se puede obtener un KDE usando una mezcla de distribuciones normales:

In [ ]:
data_1.plot.density()

Seaborn produce histogramas y gráficos de densidad de manera aún más sencilla a través del método `distplot`, el cual genera tanto un histograma como un KDE continuo (de manera simultanea). A modo de ejemplo, se consideran dos distribuciones conocidas:

In [ ]:
comp1 = np.random.normal(0, 1, size=200)
comp2 = np.random.normal(10, 2, size=200)
values = pd.Series(np.concatenate([comp1, comp2]))

In [ ]:
fig = plt.figure()
sns.distplot(values, bins=100, color='k')

## Gráficos de dispersión

Los gráficos compuestos de "puntos" o gráficos de dispersión puede ser una manera útil de examinar la relación entre datos 2-dimensionales. Por ejemplo, en los datos analizados en la `clase 1` se hace uso de tal ventaja:

**Ejercicio**

* Cargue los datos de la clase 1 './datos/C1/SCI_HDI.npy' en un DataFrame de pandas con columnas `SCI` y `HDI` respectivamente. Utilice la función `sns.regplot()` sobre estos datos.

In [ ]:
datos=np.load('./datos/C1/SCI_HDI.npy')
datos = pd.DataFrame(datos, columns=['SCI', 'HDI'])
datos.head()

In [ ]:
fig = plt.figure()
sns.regplot('SCI', 'HDI',data=datos)

Como pudo haber observado, tal función ajusta una regresión lineal sobre un gráfico de dispersión.

Otra herramienta importante en análisis de datos es la opción de analizar las relaciones entre las distintas variables que intervienen en el fenómeno que se estudia. Esto se puede hacer mediante el estudio de *gráficos de pares* o una **matriz de gráficos de dispersión**. Es posible generar gráficos de con tales características desde cero usando matplotlib, sin embargo, seaborn posee la función **pairplot** que soporta histogramas y KDE's de cada variable en la diagonal de la *matriz*.

In [ ]:
sns.pairplot(datos, diag_kind='kde')

Adicionalmente, es posible ingresar argumentos en forma de diccionario a través de la opción `plots_kws`, por ejemplo, si se quisiera agregar cierta transparencia: 
```python
sns.pairplot(arg_1, arg_2,...,plot_kws={'alpha': 0.2})
```

## Grillas y datos categóricos

Una manera de visualizar datos con múltiples variables categóricas es usando un "grilla de facetas". En seabron existe la función `factorplot` que simplifica la tarea:

In [ ]:
# Se carga la base de ejemplo "tips" presente como
# dataset de ejemplo en seaborn.

tips = sns.load_dataset("tips")
tips['tip_pct'] = tips['tip'] / (tips['total_bill'] - tips['tip'])

sns.factorplot(x='day', y='tip_pct', hue='time', col='smoker',
               kind='bar', data=tips[tips.tip_pct < 1])

En vez de agrupar por `time` usando colores distintos colores de barras, se puede expandir una "grilla de facetas" agregando una fila por valor de `time`:

In [ ]:
sns.factorplot(x='day', y='tip_pct', row='time', col='smoker',
                  kind='bar', data=tips[tips.tip_pct < 1])

`factorplot` soporta también otros tipos de gráficos que pueden ser útiles dependiendo de la que se desee mostrar. Por ejemplo los gráficos de caja:

**Ejercicio**

* Estudie el tipo de gráfico conocido domo 'violin'.